# 実践演習7-2

Scikit learn の SVMで文書分類を行います。

## 準備

必要なライブラリ等を読み込みます。

In [1]:
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report

## データの読み込み

scipyのloadarffはStringの読み込みに対応していないので、直接ファイルを開いて読み込みます。学習データをX, yに読み込みます。

In [2]:
# 学習データ
r = re.compile("'(.*)',([01])")
X = np.empty((0,1),np.string_)
y = np.empty((0,1),np.string_)

with open('ReutersGrain-train.arff', 'r') as file:
    line = file.readline()
    while not(re.match(r"@data", line)):
        line = file.readline()
    line = file.readline()  # @data のあとの空行を読み飛ばす
    for line in file:
        m = r.search(line)
        X = np.append(X, m.group(1))
        y = np.append(y, m.group(2))

学習データの事例数を確認します。

In [3]:
X.size

1554

評価データをXe, yeに読み込みます。

In [4]:
# 評価データ
Xe = np.empty((0,1),np.string_)
ye = np.empty((0,1),np.string_)
with open('ReutersGrain-test.arff', 'r') as file:
    line = file.readline()
    while not(re.match(r"@data", line)):
        line = file.readline()
    line = file.readline()  # @data のあとの空行を読み飛ばす
    for line in file:
        m = r.search(line)
        Xe = np.append(Xe, m.group(1))
        ye = np.append(ye, m.group(2))

評価データの事例数を確認します。

In [5]:
Xe.size

604

## 文字列をベクトルに変換

[CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)を使って、文字列を単語ベクトルに変換します。

In [6]:
vectorizer = CountVectorizer(min_df=1)
vectorizer   

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

学習データ中の語彙を次元として、ベクトルに変換します。

In [7]:
Xv = vectorizer.fit_transform(X)
Xv

<1554x15180 sparse matrix of type '<class 'numpy.int64'>'
	with 117651 stored elements in Compressed Sparse Row format>

評価データは、学習データの次元に合わせて、ベクトルに変換します。

In [8]:
Xev = vectorizer.transform(Xe)
Xev

<604x15180 sparse matrix of type '<class 'numpy.int64'>'
	with 43521 stored elements in Compressed Sparse Row format>

## SVMでの学習

[SVC](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)を使います。学習データの数が少ないので、線形カーネルを用います。

In [9]:
clf = SVC(kernel='linear')
clf

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

分割学習法の場合は、[classification_report](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)に正解と識別器の出力を与えます。

In [10]:
clf = clf.fit(Xv, y)
print(classification_report(ye, clf.predict(Xev), target_names=['negative', 'positive']))

             precision    recall  f1-score   support

   negative       0.98      0.99      0.99       547
   positive       0.92      0.79      0.85        57

avg / total       0.97      0.97      0.97       604



positiveのF値が0.85となり、Wekaの0.771を上回っています。これば特徴ベクトルの次元数が違うことと、SVMの実装が若干異なることが原因だと思われます。

### 多項式カーネル

2次の多項式カーネルを試してみます。

In [11]:
clf2 = SVC(kernel='poly', degree=2)
clf2

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=2, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [12]:
clf2 = clf2.fit(Xv, y)
print(classification_report(ye, clf2.predict(Xev), target_names=['negative', 'positive']))

             precision    recall  f1-score   support

   negative       0.91      1.00      0.95       547
   positive       0.00      0.00      0.00        57

avg / total       0.82      0.91      0.86       604



C:\usr\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### rbfカーネル

rbfカーネルを試してみます。gammaの値を調整することでpositiveの性能が変わります。$\gamma$が小さいと識別面の決定に寄与する学習データが少なくなるので、比較的単純な識別面になります。反対に$\gamma$が大きいと複雑な識別面になります。  

$$K(x, x')=\exp(-\gamma\|x-x'\|^2)$$

In [13]:
clf3 = SVC(gamma=0.005)
clf3

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.005, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [14]:
clf3 = clf3.fit(Xv, y)
print(classification_report(ye, clf3.predict(Xev), target_names=['negative', 'positive']))

             precision    recall  f1-score   support

   negative       0.92      1.00      0.96       547
   positive       1.00      0.21      0.35        57

avg / total       0.93      0.93      0.90       604



### 単語ベクトルの調整

単語数を減らしてみます。

In [15]:
vectorizer = CountVectorizer(min_df=8)
Xv2 = vectorizer.fit_transform(X)
print(Xv2.shape[1])
Xev2 = vectorizer.transform(Xe)
clf = clf.fit(Xv2, y)
print(classification_report(ye, clf.predict(Xev2), target_names=['negative', 'positive']))

2195
             precision    recall  f1-score   support

   negative       0.98      0.99      0.99       547
   positive       0.87      0.84      0.86        57

avg / total       0.97      0.97      0.97       604



stop wordsを有効にしてみます。

In [16]:
vectorizer = CountVectorizer(stop_words='english')
Xv2 = vectorizer.fit_transform(X)
print(Xv2.shape[1])
Xev2 = vectorizer.transform(Xe)
clf = clf.fit(Xv2, y)
print(classification_report(ye, clf.predict(Xev2), target_names=['negative', 'positive']))

14918
             precision    recall  f1-score   support

   negative       0.98      0.99      0.98       547
   positive       0.85      0.81      0.83        57

avg / total       0.97      0.97      0.97       604



tfidfを使ってみます。

In [17]:
vectorizer = TfidfVectorizer(min_df=6)
Xv2 = vectorizer.fit_transform(X)
print(Xv2.shape[1])
Xev2 = vectorizer.transform(Xe)
clf = clf.fit(Xv2, y)
print(classification_report(ye, clf.predict(Xev2), target_names=['negative', 'positive']))

2893
             precision    recall  f1-score   support

   negative       0.97      1.00      0.99       547
   positive       1.00      0.74      0.85        57

avg / total       0.98      0.98      0.97       604

